In [5]:
import numpy as np
import cv2
import os
import pickle
import json
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.filters import threshold_multiotsu, threshold_otsu
from skimage.segmentation import watershed
from skimage.feature import peak_local_max

In [6]:
class AdaptiveEnsembleVegetationSegmentation:
    """
    Adaptive Ensemble Vegetation Segmentation (AEVS) Model
    
    This model combines:
    1. Traditional vegetation indices (ExG, ExGR, CIVE, NGRDI, GLI, VEG)
    2. Novel enhanced indices (AGDI, MVI, CVS, TVI)
    3. Multi-method thresholding (Multi-Otsu, Watershed)
    4. Unsupervised ML clustering
    5. Ensemble decision fusion
    """
    def __init__(self, save_dir="./aevs_results"):
        self.scaler = StandardScaler()
        self.rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.svm_model = SVC(kernel='rbf', probability=True, random_state=42)
        self.save_dir = save_dir
        self.model_name = "AEVS"
        self.model_version = "1.0"
        
        # Create save directory
        os.makedirs(save_dir, exist_ok=True)
        os.makedirs(os.path.join(save_dir, "masks"), exist_ok=True)
        os.makedirs(os.path.join(save_dir, "indices"), exist_ok=True)
        os.makedirs(os.path.join(save_dir, "models"), exist_ok=True)
        os.makedirs(os.path.join(save_dir, "results"), exist_ok=True)
        
        # Performance tracking
        self.results_log = []
        self.best_method = None
        self.best_accuracy = 0.0
        
def compute_indices(self, rgb_image):
    R, G, B = rgb_image[:,:,0], rgb_image[:,:,1], rgb_image[:,:,2]
    R = R.astype(np.float32) + 1e-8
    G = G.astype(np.float32) + 1e-8
    B = B.astype(np.float32) + 1e-8
    
    indices = {}

    # ExG (Excess Green)
    indices['ExG'] = 2*G - R - B
    
    # ExGR (Excess Green minus Excess Red)
    ExR = 1.4*R - G
    indices['ExGR'] = indices['ExG'] - ExR
    
    # CIVE (Color Index of Vegetation Extraction)
    indices['CIVE'] = 0.441*R - 0.811*G + 0.385*B + 18.78745
    
    # NGRDI (Normalized Green Red Difference Index)
    indices['NGRDI'] = (G - R) / (G + R)
    
    # GLI (Green Leaf Index)
    indices['GLI'] = (2*G - R - B) / (2*G + R + B)
    
    # VEG (Vegetative Index)
    indices['VEG'] = G / (R**0.667 * B**(1-0.667))
    
    # Adaptive Green Dominance Index (AGDI)
    local_variance = cv2.Laplacian(rgb_image[:,:,1], cv2.CV_64F).var()
    adaptive_factor = 1 + (local_variance / 10000)
    indices['AGDI'] = adaptive_factor * (2*G - R - B) / (G + R + B)
    
    # Multi-scale Vegetation Index (MVI)
    G_blur3 = cv2.GaussianBlur(G, (3,3), 0)
    G_blur7 = cv2.GaussianBlur(G, (7,7), 0)
    indices['MVI'] = (G + 0.5*G_blur3 + 0.25*G_blur7) / (R + B + 1e-8)
    
    # Chromatic Vegetation Strength (CVS)
    intensity = (R + G + B) / 3
    chromaticity_g = G / (R + G + B + 1e-8)
    indices['CVS'] = chromaticity_g * np.log(intensity + 1)
    
    # Texture-aware Vegetation Index (TVI)
    gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    texture = cv2.Laplacian(gray, cv2.CV_64F)
    texture_strength = np.abs(texture)
    indices['TVI'] = (G - R) / (G + R) * (1 + texture_strength/255)
    
    return indices
    
    def multi_otsu_adaptive_threshold(self, index_image, n_classes=3):
        """Multi-Otsu thresholding as an alternative to standard Otsu"""
        # Normalize the index
        index_norm = ((index_image - index_image.min()) / 
                     (index_image.max() - index_image.min()) * 255).astype(np.uint8)
        
        # Multi-Otsu thresholding
        thresholds = threshold_multiotsu(index_norm, classes=n_classes)
        
        # Create segmentation (vegetation = highest class)
        regions = np.digitize(index_norm, bins=thresholds)
        vegetation_mask = (regions == (n_classes - 1)).astype(np.uint8)
        
        return vegetation_mask, thresholds
    
    def watershed_segmentation(self, index_image):
        """Watershed-based segmentation for better boundary detection"""
        # Normalize and convert to uint8
        index_norm = ((index_image - index_image.min()) / 
                     (index_image.max() - index_image.min()) * 255).astype(np.uint8)
        
        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(index_norm, (5, 5), 0)
        
        # Find local maxima as markers
        local_maxima = peak_local_maxima(blurred, min_distance=10, threshold_abs=50)
        markers = np.zeros_like(blurred)
        markers[tuple(local_maxima.T)] = np.arange(1, len(local_maxima) + 1)
        
        # Apply watershed
        labels = watershed(-blurred, markers, mask=blurred > 30)
        
        # Convert to binary mask (non-zero regions are vegetation)
        vegetation_mask = (labels > 0).astype(np.uint8)
        
        return vegetation_mask
    
    def ensemble_thresholding(self, indices_dict):
        """Ensemble approach combining multiple thresholding methods"""
        masks = []
        
        for name, index in indices_dict.items():
            # Method 1: Multi-Otsu
            mask1, _ = self.multi_otsu_adaptive_threshold(index)
            masks.append(mask1)
            
            # Method 2: Watershed
            mask2 = self.watershed_segmentation(index)
            masks.append(mask2)
        
        # Combine masks using majority voting
        mask_stack = np.stack(masks, axis=-1)
        ensemble_mask = (np.mean(mask_stack, axis=-1) > 0.5).astype(np.uint8)
        
        return ensemble_mask
    
    def individual_index_segmentation(self, rgb_image, method='otsu'):
        all_indices = self.compute_indices(rgb_image)
        
        individual_results = {}
        
        for index_name, index_values in all_indices.items():
            if method == 'otsu':
                # Standard Otsu thresholding (like base paper)
                index_norm = ((index_values - index_values.min()) / 
                             (index_values.max() - index_values.min()) * 255).astype(np.uint8)
                threshold = threshold_otsu(index_norm)
                mask = (index_norm > threshold).astype(np.uint8)
            
            elif method == 'multi_otsu':
                # Multi-Otsu alternative
                mask, _ = self.multi_otsu_adaptive_threshold(index_values)
            
            elif method == 'watershed':
                # Watershed alternative
                mask = self.watershed_segmentation(index_values)
            
            # Apply morphological refinement
            refined_mask = self.morphological_refinement(mask)
            
            individual_results[index_name] = {
                'mask': refined_mask,
                'raw_mask': mask,
                'index_values': index_values,
                'method': method
            }
        
        return individual_results
    
    def compare_individual_methods(self, rgb_image, ground_truth=None):
        """
        Compare individual index performance (like base paper methodology)
        """
        methods = ['otsu', 'multi_otsu', 'watershed']
        comparison_results = {}
        
        for method in methods:
            individual_results = self.individual_index_segmentation(rgb_image, method)
            comparison_results[method] = individual_results
            
            if ground_truth is not None:
                # Evaluate each index
                for index_name, result in individual_results.items():
                    metrics = self.evaluate_segmentation(result['mask'], ground_truth)
                    result['metrics'] = metrics
                    
                    print(f"{method.upper()} - {index_name}: Accuracy = {metrics['accuracy']:.3f}")
        
        return comparison_results
        """Apply morphological operations to refine the mask"""
        # Remove small noise
        kernel_small = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_small)
        
        # Fill small holes
        kernel_medium = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_medium)
        
        # Remove small isolated regions
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
        min_area = 100  # Minimum area threshold
        
        refined_mask = np.zeros_like(mask)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= min_area:
                refined_mask[labels == i] = 1
                
    def save_vegetation_masks(self, masks_dict, image_name, method_name):
        """
        Save vegetation masks for next phase of research
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        for mask_type, mask in masks_dict.items():
            # Save as binary image (0-255)
            mask_filename = f"{image_name}_{method_name}_{mask_type}_{timestamp}.png"
            mask_path = os.path.join(self.save_dir, "masks", mask_filename)
            cv2.imwrite(mask_path, mask.astype(np.uint8) * 255)
            
            # Also save as numpy array for easy loading
            npy_filename = f"{image_name}_{method_name}_{mask_type}_{timestamp}.npy"
            npy_path = os.path.join(self.save_dir, "masks", npy_filename)
            np.save(npy_path, mask)
        
        print(f"Masks saved for {image_name} using {method_name}")
    
    def save_model_state(self, image_name):
        """
        Save the current model state and parameters
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        model_state = {
            'model_name': self.model_name,
            'model_version': self.model_version,
            'timestamp': timestamp,
            'scaler': self.scaler,
            'best_method': self.best_method,
            'best_accuracy': self.best_accuracy,
            'results_log': self.results_log
        }
        
        # Save model state
        model_filename = f"aevs_model_{image_name}_{timestamp}.pkl"
        model_path = os.path.join(self.save_dir, "models", model_filename)
        
        with open(model_path, 'wb') as f:
            pickle.dump(model_state, f)
        
        print(f"Model state saved: {model_filename}")
        return model_path
    
    def save_results_summary(self, results, image_name):
        """
        Save comprehensive results summary
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Convert numpy arrays to lists for JSON serialization
        json_results = {}
        for method, method_data in results.items():
            json_results[method] = {}
            for index_name, index_data in method_data.items():
                json_results[method][index_name] = {
                    'method': index_data['method'],
                    'metrics': index_data.get('metrics', {}),
                    'mask_shape': index_data['mask'].shape,
                    'mask_coverage': float(np.sum(index_data['mask']) / index_data['mask'].size)
                }
        
        # Save as JSON
        results_filename = f"results_{image_name}_{timestamp}.json"
        results_path = os.path.join(self.save_dir, "results", results_filename)
        
        with open(results_path, 'w') as f:
            json.dump(json_results, f, indent=2)
        
        print(f"Results summary saved: {results_filename}")
        return results_path
    
        return refined_mask
    
    def aevs_segmentation(self, rgb_image):
        """
        Main AEVS model segmentation method
        This is the novel ensemble approach
        """
        # Step 1: Compute all indices
        all_indices = self.compute_indices(rgb_image)
        
        # Step 2: Individual method results
        individual_otsu = self.individual_index_segmentation(rgb_image, 'otsu')
        individual_multi_otsu = self.individual_index_segmentation(rgb_image, 'multi_otsu')
        individual_watershed = self.individual_index_segmentation(rgb_image, 'watershed')
        
        # Step 3: ML-based segmentation
        ml_mask = self.unsupervised_ml_segmentation(rgb_image)
        
        # Step 4: Ensemble decision making
        all_masks = []
        
        # Collect masks from different methods
        for method_results in [individual_otsu, individual_multi_otsu, individual_watershed]:
            for index_name, result in method_results.items():
                all_masks.append(result['mask'])
        
        # Add ML mask
        all_masks.append(ml_mask)
        
        # Ensemble voting
        mask_stack = np.stack(all_masks, axis=-1)
        ensemble_mask = (np.mean(mask_stack, axis=-1) > 0.5).astype(np.uint8)
        
        # Step 5: Final refinement
        final_mask = self.morphological_refinement(ensemble_mask)
        
        return final_mask, all_indices, {
            'individual_otsu': individual_otsu,
            'individual_multi_otsu': individual_multi_otsu,
            'individual_watershed': individual_watershed,
            'ml_mask': ml_mask,
            'ensemble_mask': ensemble_mask,
            'final_mask': final_mask
        }
    
    def process_and_save_image(self, rgb_image, image_name, ground_truth=None):
        """
        Complete processing pipeline with saving functionality
        """
        print(f"\nProcessing image: {image_name}")
        print("="*50)
        
        # Step 1: Compare individual methods (like base paper)
        print("1. Individual Index Comparison (Base Paper Approach):")
        individual_comparison = self.compare_individual_methods(rgb_image, ground_truth)
        
        # Step 2: Apply AEVS model
        print("\n2. AEVS Ensemble Model:")
        aevs_mask, indices, all_masks = self.aevs_segmentation(rgb_image)
        
        # Step 3: Evaluate AEVS if ground truth available
        if ground_truth is not None:
            aevs_metrics = self.evaluate_segmentation(aevs_mask, ground_truth)
            print(f"AEVS Model Accuracy: {aevs_metrics['accuracy']:.3f}")
            
            # Track best method
            if aevs_metrics['accuracy'] > self.best_accuracy:
                self.best_accuracy = aevs_metrics['accuracy']
                self.best_method = 'AEVS_Ensemble'
        
        # Step 4: Save all masks
        masks_to_save = {
            'aevs_final': aevs_mask,
            'aevs_ensemble': all_masks['ensemble_mask'],
            'aevs_ml': all_masks['ml_mask']
        }
        
        # Add best individual masks from each method
        for method_name, method_results in individual_comparison.items():
            best_index = None
            best_acc = 0
            
            if ground_truth is not None:
                for index_name, result in method_results.items():
                    if 'metrics' in result and result['metrics']['accuracy'] > best_acc:
                        best_acc = result['metrics']['accuracy']
                        best_index = index_name
            else:
                # If no ground truth, use first index
                best_index = list(method_results.keys())[0]
            
            if best_index:
                masks_to_save[f'best_{method_name}_{best_index}'] = method_results[best_index]['mask']
        
        # Save masks
        self.save_vegetation_masks(masks_to_save, image_name, 'AEVS')
        
        # Step 5: Save indices
        indices_filename = f"indices_{image_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
        indices_path = os.path.join(self.save_dir, "indices", indices_filename)
        with open(indices_path, 'wb') as f:
            pickle.dump(indices, f)
        
        # Step 6: Save results summary
        results_summary = {
            'individual_comparison': individual_comparison,
            'aevs_results': {
                'final_mask_shape': aevs_mask.shape,
                'vegetation_coverage': float(np.sum(aevs_mask) / aevs_mask.size),
                'metrics': aevs_metrics if ground_truth is not None else None
            }
        }
        
        self.save_results_summary({'aevs': results_summary}, image_name)
        
        # Step 7: Save model state
        self.save_model_state(image_name)
        
        # Step 8: Log results
        result_entry = {
            'image_name': image_name,
            'timestamp': datetime.now().isoformat(),
            'aevs_accuracy': aevs_metrics['accuracy'] if ground_truth is not None else None,
            'vegetation_coverage': float(np.sum(aevs_mask) / aevs_mask.size)
        }
        self.results_log.append(result_entry)
        
        return aevs_mask, individual_comparison, all_masks
        
        all_indices = self.compute_indices(rgb_image)
        
        features = []
        for name, index in all_indices.items():
            features.append(index.flatten())
        
        feature_matrix = np.column_stack(features)
        
        # Add spatial coordinates as features (normalized)
        h, w = rgb_image.shape[:2]
        y_coords, x_coords = np.meshgrid(range(h), range(w), indexing='ij')
        x_norm = x_coords.flatten() / w
        y_norm = y_coords.flatten() / h
        
        # Add RGB values
        r_flat = rgb_image[:,:,0].flatten() / 255.0
        g_flat = rgb_image[:,:,1].flatten() / 255.0
        b_flat = rgb_image[:,:,2].flatten() / 255.0
        
        # Combine all features
        final_features = np.column_stack([feature_matrix, x_norm, y_norm, r_flat, g_flat, b_flat])
        
        return final_features, all_indices
    
    def unsupervised_ml_segmentation(self, rgb_image, method='kmeans'):
        """Unsupervised ML-based segmentation"""
        features, indices = self.prepare_feature_vector(rgb_image)
        
        # Normalize features
        features_scaled = self.scaler.fit_transform(features)
        
        if method == 'kmeans':
            # K-means clustering (2 clusters: vegetation vs background)
            kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
            labels = kmeans.fit_predict(features_scaled)
            
            # Determine which cluster is vegetation (assume cluster with higher green index)
            cluster_0_mean_exg = np.mean(indices['ExG'].flatten()[labels == 0])
            cluster_1_mean_exg = np.mean(indices['ExG'].flatten()[labels == 1])
            
            vegetation_cluster = 1 if cluster_1_mean_exg > cluster_0_mean_exg else 0
            vegetation_mask = (labels == vegetation_cluster).reshape(rgb_image.shape[:2]).astype(np.uint8)
            
        return vegetation_mask
    
    def hybrid_segmentation(self, rgb_image):
        """Novel hybrid approach combining multiple methods"""
        # Step 1: Compute all indices
        all_indices = self.compute_indices(rgb_image)
        
        # Step 2: Ensemble thresholding
        threshold_mask = self.ensemble_thresholding(all_indices)
        
        # Step 3: ML-based segmentation
        ml_mask = self.unsupervised_ml_segmentation(rgb_image)
        
        # Step 4: Combine both approaches
        combined_mask = np.logical_and(threshold_mask, ml_mask).astype(np.uint8)
        
        # Step 5: Morphological refinement
        final_mask = self.morphological_refinement(combined_mask)
        
        return final_mask, all_indices
    
    def evaluate_segmentation(self, predicted_mask, ground_truth_mask):
        """Evaluate segmentation performance"""
        # Flatten masks
        pred_flat = predicted_mask.flatten()
        gt_flat = ground_truth_mask.flatten()
        
        # Calculate metrics
        accuracy = accuracy_score(gt_flat, pred_flat)
        
        # Calculate IoU (Intersection over Union)
        intersection = np.sum(pred_flat & gt_flat)
        union = np.sum(pred_flat | gt_flat)
        iou = intersection / union if union > 0 else 0
        
        # Calculate precision and recall
        tp = np.sum(pred_flat & gt_flat)
        fp = np.sum(pred_flat & ~gt_flat)
        fn = np.sum(~pred_flat & gt_flat)
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            'accuracy': accuracy,
            'iou': iou,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }
    
    def visualize_results(self, rgb_image, mask, indices_dict):
        """Visualize segmentation results"""
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        
        # Original image
        axes[0,0].imshow(rgb_image)
        axes[0,0].set_title('Original Image')
        
        # Final mask
        axes[0,1].imshow(mask, cmap='gray')
        axes[0,1].set_title('Vegetation Mask')
        
        # Overlay
        overlay = rgb_image.copy()
        overlay[mask == 1] = [0, 255, 0]  # Green overlay for vegetation
        axes[0,2].imshow(overlay)
        axes[0,2].set_title('Overlay')
        axes[0,2].axis('off')
        
        # Show some key indices
        key_indices = ['ExG', 'AGDI', 'MVI', 'CVS']
        for i, idx_name in enumerate(key_indices):
            if idx_name in indices_dict:
                if i < 3:
                    axes[0,3].imshow(indices_dict[idx_name], cmap='viridis')
                    axes[0,3].set_title(f'{idx_name}')
                    axes[0,3].axis('off')
                else:
                    axes[1,i-3].imshow(indices_dict[idx_name], cmap='viridis')
                    axes[1,i-3].set_title(f'{idx_name}')
                    axes[1,i-3].axis('off')
        
        plt.tight_layout()
        plt.show()

    def morphological_refinement(self, mask):
        """Apply morphological operations to refine the mask"""
        # Remove small noise
        kernel_small = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_small)
        
        # Fill small holes
        kernel_medium = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_medium)
        
        # Remove small isolated regions
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
        min_area = 100  # Minimum area threshold
        
        refined_mask = np.zeros_like(mask)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= min_area:
                refined_mask[labels == i] = 1

# Example usage functions
def process_single_image(image_path, ground_truth_path=None, save_dir="./aevs_results"):
    """
    Process a single image with the AEVS model
    """
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Load ground truth if available
    ground_truth = None
    if ground_truth_path:
        ground_truth = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)
        ground_truth = (ground_truth > 127).astype(np.uint8)  # Binarize
    
    # Get image name
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Initialize AEVS model
    aevs_model = AdaptiveEnsembleVegetationSegmentation(save_dir=save_dir)
    
    # Process and save
    final_mask, individual_results, all_masks = aevs_model.process_and_save_image(
        image, image_name, ground_truth
    )
    
    # Visualize results
    aevs_model.visualize_results(image, final_mask, 
                                aevs_model.compute_traditional_indices(image))
    
    return final_mask, individual_results, all_masks

def batch_process_with_aevs(image_folder, ground_truth_folder=None, save_dir="./aevs_batch_results"):
    """
    Batch process images with AEVS model
    """
    aevs_model = AdaptiveEnsembleVegetationSegmentation(save_dir=save_dir)
    
    processed_images = []
    
    for image_file in os.listdir(image_folder):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"\nProcessing: {image_file}")
            
            # Load image
            image_path = os.path.join(image_folder, image_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Load ground truth if available
            ground_truth = None
            if ground_truth_folder:
                gt_file = image_file.replace('.jpg', '.png').replace('.jpeg', '.png')
                gt_path = os.path.join(ground_truth_folder, gt_file)
                if os.path.exists(gt_path):
                    ground_truth = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
                    ground_truth = (ground_truth > 127).astype(np.uint8)
            
            # Process image
            image_name = os.path.splitext(image_file)[0]
            final_mask, individual_results, all_masks = aevs_model.process_and_save_image(
                image, image_name, ground_truth
            )
            
            processed_images.append({
                'image_file': image_file,
                'final_mask': final_mask,
                'individual_results': individual_results,
                'all_masks': all_masks
            })
    
    # Generate final summary report
    generate_final_report(aevs_model, save_dir)
    
    return processed_images, aevs_model

def generate_final_report(aevs_model, save_dir):
    """
    Generate final performance report
    """
    report = {
        'model_info': {
            'name': aevs_model.model_name,
            'version': aevs_model.model_version,
            'total_images_processed': len(aevs_model.results_log)
        },
        'performance_summary': {
            'best_method': aevs_model.best_method,
            'best_accuracy': aevs_model.best_accuracy,
            'average_vegetation_coverage': np.mean([r['vegetation_coverage'] for r in aevs_model.results_log])
        },
        'detailed_results': aevs_model.results_log
    }
    
    # Save report
    report_path = os.path.join(save_dir, "final_report.json")
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"\nFinal report saved: {report_path}")
    print(f"Best Method: {aevs_model.best_method}")
    print(f"Best Accuracy: {aevs_model.best_accuracy:.3f}")
    
    return report

# Usage Examples:
"""
# Single image processing
final_mask, individual_results, all_masks = process_single_image(
    "path/to/your/image.jpg", 
    "path/to/ground_truth.png",  # Optional
    save_dir="./my_aevs_results"
)

# Batch processing
processed_images, aevs_model = batch_process_with_aevs(
    image_folder="path/to/images/",
    ground_truth_folder="path/to/ground_truths/",  # Optional
    save_dir="./batch_aevs_results"
)

# The vegetation masks will be automatically saved in:
# ./batch_aevs_results/masks/
# 
# Models and results will be saved in:
# ./batch_aevs_results/models/
# ./batch_aevs_results/results/
"""

'\n# Single image processing\nfinal_mask, individual_results, all_masks = process_single_image(\n    "path/to/your/image.jpg", \n    "path/to/ground_truth.png",  # Optional\n    save_dir="./my_aevs_results"\n)\n\n# Batch processing\nprocessed_images, aevs_model = batch_process_with_aevs(\n    image_folder="path/to/images/",\n    ground_truth_folder="path/to/ground_truths/",  # Optional\n    save_dir="./batch_aevs_results"\n)\n\n# The vegetation masks will be automatically saved in:\n# ./batch_aevs_results/masks/\n# \n# Models and results will be saved in:\n# ./batch_aevs_results/models/\n# ./batch_aevs_results/results/\n'